In [11]:
import alpaca_trade_api as tradeapi
import tensorflow as tf
import requests
import gym
import gym_anytrading

from my_secrets import API_KEY, SECRET_KEY, BASE_URL
from finrl.stable_baselines.common.vec_env import DummyVecEnv

ModuleNotFoundError: No module named 'finrl.stable_baselines'

In [9]:

df = gym_anytrading.datasets.STOCKS_GOOGL.copy()

window_size = 10
start_index = window_size
end_index = len(df)

env_maker = lambda: gym.make(
    'stocks-v0',
    df = df,
    window_size = window_size,
    frame_bound = (start_index, end_index)
)

env = DummyVecEnv([env_maker])
     

NameError: name 'DummyVecEnv' is not defined

In [8]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-05-22,198.528534,199.524521,196.196198,196.946945,196.946945,3433700
2009-05-26,196.171173,202.702698,195.195190,202.382385,202.382385,6202700
2009-05-27,203.023026,206.136139,202.607605,202.982986,202.982986,6062500
2009-05-28,204.544540,206.016022,202.507507,205.405411,205.405411,5332200
2009-05-29,206.261261,208.823822,205.555557,208.823822,208.823822,5291100
...,...,...,...,...,...,...
2018-08-23,1219.880005,1235.170044,1219.349976,1221.160034,1221.160034,1233300
2018-08-24,1226.000000,1237.400024,1221.420044,1236.750000,1236.750000,1222700
2018-08-27,1244.140015,1257.869995,1240.680054,1256.270020,1256.270020,1429000


In [6]:
# Set up Alpaca API credentials
api = tradeapi.REST(API_KEY, SECRET_KEY, api_version='v2')

# Create gym environment
env = gym.make('StockTrading-v0', data_path='<PATH TO HISTORICAL DATA>', api_key='<API KEY ID>', api_secret='<API SECRET KEY>', symbol='AAPL')

# Create and train PPO agent
agent = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(env.action_space.n)
])
ppo = tf_agents.agents.PPOAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    actor_net=agent,
    value_net=agent,
    num_epochs=10,
    num_parallel_environments=4
)

UnregisteredEnv: No registered env with id: StockTrading-v0

In [ ]:

training_data = env.get_training_data()
for i in range(1000):
    ppo.train(training_data)

# Retrieve news data
news = requests.get('https://newsapi.org/v2/top-headlines?sources=bloomberg&apiKey=<API KEY>').json()

# Preprocess news data
news_text = [article['title'] for article in news['articles']]
news_vectors = tf.vectorize_preprocess(news_text)

# Trade based on model predictions
observation = env.reset()
while True:
    action_step = ppo.policy.action(observation)
    observation, reward, done, info = env.step(action_step.action)
    if done:
        break

# Schedule script to run every minute